In [ ]:
import pandas as pd
import numpy as np
import os
os.chdir('/project/jevans/MAG_Dec_2021_snapshot')
from tqdm import tqdm
tqdm.pandas()

import json
# from langdetect import detect

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False,nb_workers=15)

import gc

import nltk

In [ ]:
import pickle

def clean_id(x):
    try:
        return int(x)
    except:
        return np.nan

for seq in range(1,11):
    space=Ulist[seq-1]
    period_chunk=pd.read_csv(f'/project/jevans/likun/science/building_science_space/period_corpus/{seq}_tokenized.csv',low_memory=False,header=None,lineterminator='\n')
    period_chunk.columns=['index','ID','year','cleaned','tokens']
    period_chunk.drop_duplicates(inplace=True)
    period_chunk['year_group']=period_chunk['year'].apply(lambda x: get_year_group(x))
    period_chunk['tokens']=period_chunk['tokens'].progress_apply(lambda x: get_tokens(x))
    period_chunk['index']=period_chunk['tokens'].apply(lambda x: [w2i[n] for n in x if n in w2i.keys()])
    period_chunk['vect']=period_chunk['index'].apply(lambda x: np.sum(space[x],axis=0))
    period_chunk['word_count']=period_chunk['index'].apply(lambda x: len(x))
    period_chunk['ID']=period_chunk['ID'].apply(lambda x: clean_id(x))
    period_chunk=period_chunk[~period_chunk['ID'].isin(other_items)]
    period_chunk=pd.merge(period_chunk,author_df,how='inner',on='ID')
    for year in list(set(period_chunk['year'])):
        period_year=period_chunk[period_chunk['year']==year]
        with open(f'/project/jevans/likun/science_team/yearly_paper_vect/yearly_vects_{year}.pickle','wb') as handle:
            pickle.dump(period_year[['author_id','ID','year','year_group','index','vect','word_count']], handle, protocol=pickle.HIGHEST_PROTOCOL)


100%|██████████| 10392439/10392439 [06:04<00:00, 28544.87it/s]


In [ ]:
import pickle as pkl
from collections import OrderedDict
from sklearn.metrics.pairwise import cosine_distances as cosine_distance
from sklearn.metrics.pairwise import cosine_similarity

hash_=pd.read_csv('/project/jevans/likun/science/building_science_space/period_corpus/ten_period_version/words_hash_150_10period_version.csv')
Ulist=pkl.load(open('/project/jevans/likun/science/building_science_space/period_corpus/ten_period_version/L10T10G100A1ngU_iter9.p','rb'))


def get_vect(index_lst,space):
    if len(index_lst)>0:
        vect=space[index_lst]
        return np.mean(vect,axis=0)
    else:
        return np.nan

In [ ]:
def get_perspective_diversity(perspective_vectors):
    bad_loc=np.where(np.isnan(perspective_vectors))
    perspective_vectors=np.delete(perspective_vectors,bad_loc[0],0)

    if len(perspective_vectors)>0:
        bad_loc=np.where(np.isnan(perspective_vectors))
        perspective_vectors=np.delete(perspective_vectors,bad_loc[0],0)

        N=len(perspective_vectors)
        perspective_diversity_matrix=cosine_distance(perspective_vectors, perspective_vectors)
        perspective_diversity_matrix=(1-np.eye(N))*perspective_diversity_matrix
        perspective_diversity=np.nansum(perspective_diversity_matrix)/(N**2-N)

        return perspective_diversity
    else:
        return np.nan,np.nan,np.nan

def get_background_diversity(inventor_vector):
    bad_loc=np.where(np.isnan(inventor_vector))
    inventor_vector=np.delete(inventor_vector,bad_loc[0],0)

    N=len(inventor_vector)
    distances=cosine_distance(inventor_vector,inventor_vector)
    distances=(1-np.eye(N))*distances
    background_diversity=np.nansum(distances)/(N**2-N)
    return background_diversity

def get_distance(arr1,arr2):
    try:
        return cosine_distance([arr1],[arr2])[0][0]
    except:
        return np.nan

In [ ]:
##example

import pickle as pkl
from collections import OrderedDict
from sklearn.metrics.pairwise import cosine_distances as cosine_distance
from sklearn.metrics.pairwise import cosine_similarity

team_empirical=np.array([[7,5],[9,5],[8,4],[8,8]])
## assuming central project is at [8,5]
pv_empirical=np.array([8,5])-team_empirical
print(get_perspective_diversity(pv_empirical))
print(get_background_diversity(team_empirical))


1.3333333333333333
0.020495742000445388


In [ ]:
##in this code, "ID" is the id for papers/projects, and author_id is the id for team members.

for year in range(1981,2016):
    year_group=get_year_group(year)
    space=Ulist[year_group-1]

    with open(f'/project/jevans/likun/science_team/yearly_paper_vect/yearly_vects_{year}.0.pickle','rb') as file:
        target_year=pkl.load(file)
    target_authors=list(set(target_year['author_id']))
    target_year['author_id']=target_year['author_id'].apply(lambda x: str(x).strip())

    history_dfs=pd.DataFrame()
    for hy in tqdm(range(year-5,year)):
        with open(f'/project/jevans/likun/science_team/yearly_paper_vect/yearly_vects_{hy}.0.pickle','rb') as file:
            history_part=pkl.load(file)
            history_part=history_part[history_part['author_id'].isin(target_authors)]
        history_dfs=pd.concat([history_dfs,history_part])
    history_dfs=history_dfs[['author_id','ID','year','index']]
    history_dfs['author_id']=history_dfs['author_id'].apply(lambda x: str(x).strip())

    history_dfs=history_dfs[~history_dfs.astype(str).duplicated()]
    history_dfs=history_dfs[['author_id','year','index']]

    history_dfs['year']=history_dfs['year'].apply(lambda x: int(float(x)))
    gc.collect()

    history_dfs=pd.DataFrame(history_dfs.groupby('author_id')['index'].sum())
    history_dfs.columns=['all_index']
    target_year=pd.merge(target_year,history_dfs,how='left',on='assigned_id')
    target_year['vect']=target_year['index'].parallel_apply(lambda x: get_vect(x,space))

    target_year=target_year[~target_year['all_index'].isna()]
    target_year['history_vect']=target_year['all_index'].parallel_apply(lambda x: get_vect(x,space))
    target_year['perspective_vect']=target_year['vect']-target_year['history_vect']

    target_year['perspective_vect']=target_year['perspective_vect'].apply(lambda x: [list(x)])
    perspective_sets=pd.DataFrame(target_year.groupby('ID')['perspective_vect'].sum())
    perspective_sets.reset_index(inplace=True)
    perspective_sets['perspective_vect']=perspective_sets['perspective_vect'].apply(lambda x: np.array(x))
    perspective_sets['perspective_diversity']=perspective_sets['perspective_vect'].parallel_apply(lambda x: get_perspective_diversity(x))
    perspective_sets=perspective_sets[['ID','perspective_diversity']]

    target_year=target_year[~target_year['history_vect'].isna()]
    target_year['history_vect']=target_year['history_vect'].apply(lambda x: [list(x)])
    background_sets=pd.DataFrame(target_year.groupby('ID')['history_vect'].sum())
    background_sets.reset_index(inplace=True)
    background_sets['history_vect']=background_sets['history_vect'].apply(lambda x: np.array(x))
    background_sets['bdiversity']=background_sets['history_vect'].parallel_apply(lambda x: get_background_diversity(x))
    background_sets=background_sets[['ID','bdiversity']]

    team_size=pd.DataFrame(target_year.groupby('ID')['author_id'].count())
    team_size.columns=['team_size']
    team_size.reset_index(inplace=True)

    outcomes=pd.merge(team_size,perspective_sets,how='left',on='ID')
    outcomes=pd.merge(outcomes,background_sets,how='left',on='ID')
    outcomes=outcomes[outcomes['team_size']>1]
    outcomes['year']=year
    outcomes.to_csv(f'/project/jevans/likun/science_team/scores_results/{year}_scores_5y.csv')
    gc.collect()

100%|██████████| 5/5 [04:47<00:00, 57.52s/it]


2013 unmatched: 26296


100%|██████████| 5/5 [12:32<00:00, 150.49s/it]


2014 unmatched: 28033


100%|██████████| 5/5 [12:34<00:00, 150.95s/it]


2015 unmatched: 28408
